In [1]:
import breakwater as bw
import pandas as pd
import os
from pathlib import Path
import numpy as np

In [2]:
from development_overtopping_DKA import (eurotop2018_6_5, 
                                         surf_similarity, 
                                         gamma_beta_eurotop_2018_6_9, 
                                         calc_beta)
from development_armour_stability_DKA import (vangent_armour_reduction, 
                                              hudson_fixed_slope, 
                                              rock_manual_5_196, 
                                              rock_manual_5_195, 
                                              rock_manual_5_194)
from development_material_DKA import(get_class,
                                     get_Dn50)

from development_scour_DKA import(sumer_fredsoe,
                                 v_scour)


from breakwater.utils.exceptions import user_warning

In [3]:
project_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                             index_col = 1,
                             sheet_name='Input_Project specific')
requirements_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                                  index_col = 0,
                                  sheet_name='Input_requirements')
wave_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                          index_col = 0,
                          sheet_name='input_hydrotechnical',
                         skiprows = 1)
cross_section_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                   sheet_name='Input_Cross section',
                                  skiprows = 1)
concrete_element_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                      sheet_name='Input_concrete_elements',
                                      index_col = 0,
                                      skiprows = 1)
gradings_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                      sheet_name='input_rock_gradings',
                                      index_col = 0,
                                      skiprows = 2)

C:\ProgramData\anaconda3\envs\env_bw\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
#project_data

In [5]:
#requirements_data

In [6]:
#wave_data

In [7]:
#cross_section_data

In [8]:
#gradings_data

In [ ]:
# VALIDATE CREST HEIGHT FOR OVERTOPPING

# Do we want to put this entire part in a function to keep the scripts clean a bit?


Calculation_case = 57
Cross_section_id = 0
# Public access: allowed

# Open project specific parameters
g = project_data.at['g', 'Value']

# Open structure specific parameters
tana          = cross_section_data.at[Cross_section_id, 'tan_a']
dir_structure = cross_section_data.at[Cross_section_id, 'dir_structure']
safety        = cross_section_data.at[Cross_section_id, 'safety']

# Get info for sea state. Currently implemented to do only the first sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
wl       = wave_data.at[Calculation_case, 'wl']
Tm_min_1 = wave_data.at[Calculation_case, 'Tm-1,0']
dir_wave = wave_data.at[Calculation_case, 'dir_wave']

# Get allowed q for the location and calculation case
LS        = wave_data.at[Calculation_case, 'Limit State']
q_allowed = requirements_data.at['Overtopping limit public access', LS]


#Calculate roughness reduction
# Set armour for rougness reduction
armour_layer = 'Rock'
layers       = 2
permeability = 'permeable'

# Calculate surf-similarity with Hm0 and Tm-1,0
xi_m_min_1 = surf_similarity(tana, Hm0, Tm_min_1, g)

gamma_f = bw.core.overtopping.gamma_f(armour_layer, 
                                      xi_m_min_1, 
                                      layers = layers, 
                                      permeability = permeability) 

#Calculate obliqueness reduction. Function based on SAWP-#3504459-V48-IHS-COA-xxx-CAL_Armour_Stability_under_Waves.XLSM
beta = calc_beta(dir_structure, dir_wave)
gamma_beta = gamma_beta_eurotop_2018_6_9(beta)


Rc = eurotop2018_6_5(g, Hm0, q_allowed, gamma_f, gamma_beta, limit = False, safety=safety)
z_crest = Rc + wl